In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import statsmodels.api as sm
import geopandas as gpd
import os
import dask
import dask.dataframe as dd
import itertools
from itertools import chain
from math import sqrt, floor, ceil, isnan
import multiprocess
import multiprocessing
import importlib
from importlib import reload
from collections import Counter
from fuzzywuzzy import process, fuzz
import time
import warnings
import datetime
from datetime import datetime
from datetime import date
warnings.filterwarnings("error")

pd.options.display.max_columns = 500
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 400

try:
    del(FUN_proc_name)
except:
    pass
import FUN_proc_name
importlib.reload(FUN_proc_name)
from FUN_proc_name import FUN_proc_name


# 1. Clean SNL (Now S&P) Data

In [2]:
SNL = pd.DataFrame()
for year in range(1980,2024):
    SNL_oneyear = pd.read_excel("../RawData/SNL/"+str(year)+".xls",skiprows=list(range(0,14)))
    SNL_oneyear = SNL_oneyear[~pd.isnull(SNL_oneyear['Transaction Id'])]
    SNL_oneyear = SNL_oneyear[SNL_oneyear['Transaction Id']!=\
        'S&P Credit Ratings and Research provided by S&P Global Ratings']
    SNL_oneyear = SNL_oneyear[SNL_oneyear['Transaction Id']!=\
        '* Indicates that the deal directly or indirectly involved a private equity investor.']
    SNL_oneyear = SNL_oneyear[SNL_oneyear['Transaction Id']!=\
        '1 Transaction data supplemented from Crunchbase.']
    SNL = pd.concat([SNL,SNL_oneyear])

SNL['Target_raw'] = SNL['Target or Issuer']
SNL['Target'] = SNL['Target_raw'].str.upper()
SNL['Buyer_raw'] = SNL['Buyer']
SNL['Buyer'] = SNL['Buyer_raw'].str.upper()
SNL = SNL[~pd.isnull(SNL['Target'])]
SNL = SNL[~pd.isnull(SNL['Buyer'])]
SNL['Target'] = SNL['Target'].apply(FUN_proc_name)
SNL['Buyer'] = SNL['Buyer'].apply(FUN_proc_name)
SNL['Target'] = SNL['Target'].str.replace('*','',regex=False)
SNL['Buyer'] = SNL['Buyer'].str.replace('*','',regex=False)

# Remove institutions in businesses not of interest to me
SNL = SNL[SNL['Primary Industry (CIQ/GICS)']!='Insurance Brokers']
SNL = SNL[SNL['Primary Industry (CIQ/GICS)']!='Property and Casualty Insurance']
SNL = SNL[SNL['Primary Industry (CIQ/GICS)']!='Life and Health Insurance']
SNL = SNL[SNL['Primary Industry (CIQ/GICS)']!='Reinsurance']
SNL = SNL[SNL['Primary Industry (CIQ/GICS)']!='Multi-line Insurance']

# 2. Construct a sample of withdrawn M&A

In [3]:
# Restrict to SNL or SDC deals where both sides are in GPF

GPF_names = pd.read_parquet('../CleanData/SDC/0H_GPF_Parent.parquet')
MA = pd.read_csv("../CleanData/SDC/0B_MA_SDC.csv")
GPF_names = GPF_names[['name_GPF']].drop_duplicates()

SNL_withdrawn = SNL[SNL['Status']=='Terminated/Withdrawn']
MA_withdrawn = MA[MA['Status']=='Withdrawn'].copy()

In [4]:
SNL_withdrawn = SNL_withdrawn[['Target','Buyer','Completion Date','Announcement Date']]

# Based on fuzzy matching results, edit names in SNL (of withdrawn M&A banks) so it is consistent with GPF
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FEDERATED BANK SSB','Target'] = 'FEDERATED BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='AMERICAN FEDERAL S&LA','Target'] = 'AMERICAN FEDERAL S&L ASSOC'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FIRST FEDERAL SAVINGS BANK OF GEORGIA','Target'] = 'FIRST FEDERAL SAVINGS BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='INDEPENDENT BANK GROUP','Target'] = 'INDEPENDENT BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='COWEN GROUP','Target'] = 'COWEN'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='REFCO','Target'] = 'REFCO PARTNERS'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='CITIZENS FEDERAL S&LA','Target'] = 'CITIZENS FEDERAL SVG & LOGAN'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='WACHOVIA','Target'] = 'WACHOVIA BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='THE TORONTO DOMINION BANK','Target'] = 'TD BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='LEHMAN BROTHERS HOLDINGS','Target'] = 'LEHMAN BROTHERS'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='METROPOLITAN BANK HOLDING','Target'] = 'METROPOLITAN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FIRST MID ILLINOIS BANK','Target'] = 'FIRST MID ILLINOIS BANK & TR'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='BROADWAY FINANCIAL INVESTMENTS','Target'] = 'BROADWAY FINANCIAL'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='COMMERCE BANC','Target'] = 'COMMERCE BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='BANCFIRST OHIO','Target'] = 'BANCFIRST'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='METROPOLITAN FEDERAL BANK FSB','Target'] = 'METROPOLITAN FEDERAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='BOONE COUNTY NATIONAL BANK OF COLUMBIA','Target'] = 'BOONE COUNTY NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='COLONIAL BANCGROUP','Target'] = 'COLONIAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='TRUSTCO BANK CORP NY','Target'] = 'TRUSTCO BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='MICHIGAN NATIONAL','Target'] = 'MICHIGAN NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FIRST NATIONAL BANK IN TAYLORVILLE','Target'] = 'FIRST NATIONAL BANK (IN)'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='TRANSAMERICA PUBLIC FINANCE','Target'] = 'TRANSAMERICA'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='MERIDIAN BANK NEW JERSEY','Target'] = 'MERIDIAN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='BB&T FINANCIAL','Target'] = 'BB&T'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='MERCHANTS NATIONAL','Target'] = 'MERCHANTS NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='CCNB','Target'] = 'CCNB BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='CHASE MANHATTAN CAPITAL','Target'] = 'CHASE MANHATTAN'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FIDELITY NATIONAL','Target'] = 'FIDELITY NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='UNION PLANTERS','Target'] = 'UNION PLANTERS NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='EQUITY BANK A','Target'] = 'EQUITY BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='SIGNET BANK RICHMOND','Target'] = 'SIGNET BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='SOUTHTRUST','Target'] = 'SOUTHTRUST SECURITIES'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='PACIFIC WESTERN BANKLOS ANGELES','Target'] = 'PACIFIC WESTERN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='HOME SAVINGS & LOAN FA','Target'] = 'HOME SAVINGS & LOAN'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FIRST NATIONAL BANK HOLDING','Target'] = 'FIRST NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='SOUTHWEST SECURITIES GROUP','Target'] = 'SOUTHWEST SECURITIES'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='BARNETT BANKS TRUST','Target'] = 'BARNETT BANKS'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='CITIZENS & NORTHERN','Target'] = 'CITIZENS & NORTHERN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='NORWEST','Target'] = 'NORWEST INVESTMENT SERVICES'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='EVERGREEN BANK GROUP','Target'] = 'EVERGREEN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='SUNTRUST BANKS','Target'] = 'SUNTRUST BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='NBC BANK OF OKLAHOMA','Target'] = 'NBC BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='INTEGRITY BANK SSB','Target'] = 'INTEGRITY BANK'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='FIRST AMERICAN CAPITAL','Target'] = 'FIRST AMERICAN CAPITAL MARKETS'
SNL_withdrawn.loc[SNL_withdrawn['Target']=='TRANSAMERICA PUBLIC FINANCE','Target'] = 'TRANSAMERICA'

SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FEDERATED BANK SSB','Buyer'] = 'FEDERATED BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='AMERICAN FEDERAL S&LA','Buyer'] = 'AMERICAN FEDERAL S&L ASSOC'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FIRST FEDERAL SAVINGS BANK OF GEORGIA','Buyer'] = 'FIRST FEDERAL SAVINGS BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='INDEPENDENT BANK GROUP','Buyer'] = 'INDEPENDENT BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='COWEN GROUP','Buyer'] = 'COWEN'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='REFCO','Buyer'] = 'REFCO PARTNERS'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='CITIZENS FEDERAL S&LA','Buyer'] = 'CITIZENS FEDERAL SVG & LOGAN'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='WACHOVIA','Buyer'] = 'WACHOVIA BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='THE TORONTO DOMINION BANK','Buyer'] = 'TD BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='LEHMAN BROTHERS HOLDINGS','Buyer'] = 'LEHMAN BROTHERS'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='METROPOLITAN BANK HOLDING','Buyer'] = 'METROPOLITAN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FIRST MID ILLINOIS BANK','Buyer'] = 'FIRST MID ILLINOIS BANK & TR'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='BROADWAY FINANCIAL INVESTMENTS','Buyer'] = 'BROADWAY FINANCIAL'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='COMMERCE BANC','Buyer'] = 'COMMERCE BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='BANCFIRST OHIO','Buyer'] = 'BANCFIRST'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='METROPOLITAN FEDERAL BANK FSB','Buyer'] = 'METROPOLITAN FEDERAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='BOONE COUNTY NATIONAL BANK OF COLUMBIA','Buyer'] = 'BOONE COUNTY NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='COLONIAL BANCGROUP','Buyer'] = 'COLONIAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='TRUSTCO BANK CORP NY','Buyer'] = 'TRUSTCO BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='MICHIGAN NATIONAL','Buyer'] = 'MICHIGAN NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FIRST NATIONAL BANK IN TAYLORVILLE','Buyer'] = 'FIRST NATIONAL BANK (IN)'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='TRANSAMERICA PUBLIC FINANCE','Buyer'] = 'TRANSAMERICA'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='MERIDIAN BANK NEW JERSEY','Buyer'] = 'MERIDIAN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='BB&T FINANCIAL','Buyer'] = 'BB&T'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='MERCHANTS NATIONAL','Buyer'] = 'MERCHANTS NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='CCNB','Buyer'] = 'CCNB BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='CHASE MANHATTAN CAPITAL','Buyer'] = 'CHASE MANHATTAN'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FIDELITY NATIONAL','Buyer'] = 'FIDELITY NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='UNION PLANTERS','Buyer'] = 'UNION PLANTERS NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='EQUITY BANK A','Buyer'] = 'EQUITY BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='SIGNET BANK RICHMOND','Buyer'] = 'SIGNET BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='SOUTHTRUST','Buyer'] = 'SOUTHTRUST SECURITIES'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='PACIFIC WESTERN BANKLOS ANGELES','Buyer'] = 'PACIFIC WESTERN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='HOME SAVINGS & LOAN FA','Buyer'] = 'HOME SAVINGS & LOAN'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FIRST NATIONAL BANK HOLDING','Buyer'] = 'FIRST NATIONAL BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='SOUTHWEST SECURITIES GROUP','Buyer'] = 'SOUTHWEST SECURITIES'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='BARNETT BANKS TRUST','Buyer'] = 'BARNETT BANKS'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='CITIZENS & NORTHERN','Buyer'] = 'CITIZENS & NORTHERN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='NORWEST','Buyer'] = 'NORWEST INVESTMENT SERVICES'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='EVERGREEN BANK GROUP','Buyer'] = 'EVERGREEN BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='SUNTRUST BANKS','Buyer'] = 'SUNTRUST BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='NBC BANK OF OKLAHOMA','Buyer'] = 'NBC BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='INTEGRITY BANK SSB','Buyer'] = 'INTEGRITY BANK'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='FIRST AMERICAN CAPITAL','Buyer'] = 'FIRST AMERICAN CAPITAL MARKETS'
SNL_withdrawn.loc[SNL_withdrawn['Buyer']=='TRANSAMERICA PUBLIC FINANCE','Buyer'] = 'TRANSAMERICA'


In [5]:
%%script false --no-raise-error

# Note that outputs of this fuzzy match has been examined and based on it names in withdrawn SNL has been edited to match with GPF

def proc_list(SNL_withdrawn):

    fuzzy_matches_targets = []
    for idx,row in SNL_withdrawn.iterrows():
        ratio = process.extract(row['Target'],GPF_names['name_GPF'],limit=3,scorer=fuzz.partial_ratio)
        for item in ratio:
            if item[1]>=90:
                fuzzy_matches_targets = fuzzy_matches_targets+[{
                    'Target':row['Target'],
                    'name_GPF':item[0],
                    'score':item[1],
                }]
    fuzzy_matches_targets = pd.DataFrame(fuzzy_matches_targets)
    
    return fuzzy_matches_targets

SNL_withdrawn_dd = dd.from_pandas(SNL_withdrawn, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    fuzzy_matches_targets = SNL_withdrawn_dd.map_partitions(proc_list, 
    meta=pd.DataFrame(columns=['Target','name_GPF','score'])).compute()

def proc_list(SNL_withdrawn):

    fuzzy_matches_acquirers = []
    for idx,row in SNL_withdrawn.iterrows():
        ratio = process.extract(row['Buyer'],GPF_names['name_GPF'],limit=3,scorer=fuzz.partial_ratio)
        for item in ratio:
            if item[1]>=90:
                fuzzy_matches_acquirers = fuzzy_matches_acquirers+[{
                    'Buyer':row['Buyer'],
                    'name_GPF':item[0],
                    'score':item[1],
                }]
    fuzzy_matches_acquirers = pd.DataFrame(fuzzy_matches_acquirers)
    
    return fuzzy_matches_acquirers

SNL_withdrawn_dd = dd.from_pandas(SNL_withdrawn, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    fuzzy_matches_acquirers = SNL_withdrawn_dd.map_partitions(proc_list, 
    meta=pd.DataFrame(columns=['Buyer','name_GPF','score'])).compute()


In [6]:
MA_withdrawn.loc[MA_withdrawn['target']=='SECURITY NATIONAL','target'] = 'SECURITY NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST INTERSTATE BANKCA','target'] = 'FIRST INTERSTATE BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='WELLS FARGO CAPITAL C','target'] = 'WELLS FARGO'
MA_withdrawn.loc[MA_withdrawn['target']=='BACHE HALSEY STUART SHIELDS','target'] = 'BACHE HALSEY STUART SHIELDS IN'
MA_withdrawn.loc[MA_withdrawn['target']=='CONTINENTAL ILLINOIS','target'] = 'CONTINENTAL ILLINOIS NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='LF ROTHSCHILD HOLDINGS','target'] = 'LF ROTHSCHILD'
MA_withdrawn.loc[MA_withdrawn['target']=='BANK OF BOSTON CORPBOSTONMA','target'] = 'BANK OF BOSTON'
MA_withdrawn.loc[MA_withdrawn['target']=='COLONIAL BANCGROUP','target'] = 'COLONIAL BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='PONTIAC STATE BANK PONTIAC','target'] = 'PONTIAC STATE BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='COMMERCIAL BANK & TR','target'] = 'COMMERCIAL BANK & TRUST'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST CAPITAL','target'] = 'FIRST CAPITAL BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='HERITAGE BANK NJ','target'] = 'HERITAGE BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='SECURITY NATIONAL','target'] = 'SECURITY NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST NATIONAL STATE BANK','target'] = 'FIRST NATIONAL STATE BANK NJ'
MA_withdrawn.loc[MA_withdrawn['target']=='MELLON NATIONAL','target'] = 'MELLON NATIONAL BANK & TR'
MA_withdrawn.loc[MA_withdrawn['target']=='UNITED JERSEY BANKSPRINCETONNJ','target'] = 'UNITED JERSEY BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='BANC ONE CORPCOLUMBUSOHIO','target'] = 'BANC ONE'
MA_withdrawn.loc[MA_withdrawn['target']=='BANK OF NEW ENGLANDBOSTONMA','target'] = 'BANK OF NEW ENGLAND'
MA_withdrawn.loc[MA_withdrawn['target']=='AMERITRUST CORPCLEVELANDOH','target'] = 'AMERITRUST'
MA_withdrawn.loc[MA_withdrawn['target']=='CENTRAL FIDELITY BANKS INCVA','target'] = 'CENTRAL FIDELITY BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='WESTAMERICA BANKCALIFORNIA','target'] = 'WESTAMERICA BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='NORTH FORK BANKMELVILLENY','target'] = 'NORTH FORK BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='HUDSON UNITED BANKNJ','target'] = 'HUDSON UNITED BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='DIME BANK INCNEW YORKNY','target'] = 'DIME BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='MICHIGAN NATIONAL','target'] = 'MICHIGAN NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='COMERICA','target'] = 'COMERICA BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST OF MICHIGAN CAPITAL','target'] = 'FIRST OF MICHIGAN'
MA_withdrawn.loc[MA_withdrawn['target']=='PIEDMONT','target'] = 'PIEDMONT BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='NORTHEAST BANKME','target'] = 'NORTHEAST BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='PROGRESSIVE SECURITIES','target'] = 'PROGRESSIVE'
MA_withdrawn.loc[MA_withdrawn['target']=='MIDLANTIC BANKS INCEDISONNJ','target'] = 'MIDLANTIC BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='TRUSTCO BANK CORP NY','target'] = 'TRUSTCO BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='TROY FINANCIAL CORPTROYNY','target'] = 'TROY FINANCIAL'
MA_withdrawn.loc[MA_withdrawn['target']=='KEY STATE BANK OWOSSO','target'] = 'KEY STATE BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='QUICK & REILLY GROUP','target'] = 'QUICK & REILLY'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST PENNSYLVANIA BANK','target'] = 'FIRST PENNSYLVANIA'
MA_withdrawn.loc[MA_withdrawn['target']=='CENTERRE BANKST LOUIS','target'] = 'CENTERRE BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST FIDELITY BANKNJ','target'] = 'FIRST FIDELITY BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='HSBC HONG KONG','target'] = 'HSBC'
MA_withdrawn.loc[MA_withdrawn['target']=='PNC FINANCIALPITTSBURGHPA','target'] = 'PNC BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='DELAWARE TRUST CODE','target'] = 'DELAWARE TRUST'
MA_withdrawn.loc[MA_withdrawn['target']=='AMSOUTH BANKALABAMA','target'] = 'AMSOUTH BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='EF HUTTON GROUP','target'] = 'E F HUTTON'
MA_withdrawn.loc[MA_withdrawn['target']=='DEAN WITTER FINANCIAL SERVICES','target'] = 'DEAN WITTER'
MA_withdrawn.loc[MA_withdrawn['target']=='DIME BANKMARIETTAOHIO','target'] = 'DIME BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='NATIONAL BANK OF COMMERCEDC','target'] = 'NATIONAL BANK OF COMMERCE'
MA_withdrawn.loc[MA_withdrawn['target']=='A BROWN & SONS','target'] = 'ALEX BROWN & SONS'
MA_withdrawn.loc[MA_withdrawn['target']=='KEMPER','target'] = 'KEMPER SECURITIES'
MA_withdrawn.loc[MA_withdrawn['target']=='CONNECTICUT BANK&TRHARTFORDCT','target'] = 'CONNECTICUT BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='TORONTO DOMINION BANK','target'] = 'TD BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST HORIZON','target'] = 'FIRST HORIZON BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='KIDDER PEABODY GROUP(GEN ELEC)','target'] = 'KIDDER PEABODY'
MA_withdrawn.loc[MA_withdrawn['target']=='MID AMERICA NATIONAL BANK','target'] = 'MID AMERICAN NATIONAL BANK & TR'
MA_withdrawn.loc[MA_withdrawn['target']=='GREAT SOUTHERN BANKLANTANAFL','target'] = 'GREAT SOUTHERN BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='RODMAN & RENSHAW CAPITAL GROUP','target'] = 'RODMAN & RENSHAW'
MA_withdrawn.loc[MA_withdrawn['target']=='STERLING BANK CO FSB','target'] = 'STERLING BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='PROVIDENT BANK CORPMD','target'] = 'PROVIDENT BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FRONTIER BANK NALA PALMACA','target'] = 'FRONTIER BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='MARK TWAIN BANKMO','target'] = 'MARK TWAIN BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='ST JAMES BANKLUTCHERLA','target'] = 'ST JAMES BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='REGIONS FINANCIAL CORPAL','target'] = 'REGIONS BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST COMMONWEALTH BANKWISEVA','target'] = 'FIRST COMMONWEALTH BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='PREMIER BANKSHSBLUEFIELDVA','target'] = 'PREMIER BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='BANK PLUS CORPLOS ANGELESCA','target'] = 'BANK PLUS'
MA_withdrawn.loc[MA_withdrawn['target']=='CHASE MANHATTAN CORPNY','target'] = 'CHASE MANHATTAN'
MA_withdrawn.loc[MA_withdrawn['target']=='NATIONAL CITY BANK INCIN','target'] = 'NATIONAL CITY BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='CNB BANK INCIN','target'] = 'CNB BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='HSBC HOLDINGS','target'] = 'HSBC'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST SECURITY CORPUTAH','target'] = 'FIRST SECURITY'
MA_withdrawn.loc[MA_withdrawn['target']=='STURGIS BANK&TRUSTTHREE','target'] = 'STURGIS BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='SOUTHERN MICHIGAN BANKMI','target'] = 'SOUTHERN MICHIGAN BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='GOLDMAN SACHS GROUP','target'] = 'GOLDMAN SACHS'
MA_withdrawn.loc[MA_withdrawn['target']=='BSB BANK INCBINGHAMTONNY','target'] = 'BSB BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST WESTERN BANKNC','target'] = 'FIRST WESTERN BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='AG EDWARDS','target'] = 'A G EDWARDS & SONS'
MA_withdrawn.loc[MA_withdrawn['target']=='LADENBURG THALMANN FINL SERVICES','target'] = 'LADENBURG THALMANN'
MA_withdrawn.loc[MA_withdrawn['target']=='FIRST BANK OF MISSOURI BANK','target'] = 'FIRST BANK OF MISSOURI'
MA_withdrawn.loc[MA_withdrawn['target']=='STERLING BANK HOUSTON TEXAS','target'] = 'STERLING BANK'
MA_withdrawn.loc[MA_withdrawn['target']=='TRI VALLEY BANKSAN RAMONCA','target'] = 'TRI VALLEY BANK'

MA_withdrawn.loc[MA_withdrawn['acquiror']=='SECURITY NATIONAL','acquiror'] = 'SECURITY NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST INTERSTATE BANKCA','acquiror'] = 'FIRST INTERSTATE BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='WELLS FARGO CAPITAL C','acquiror'] = 'WELLS FARGO'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='BACHE HALSEY STUART SHIELDS','acquiror'] = 'BACHE HALSEY STUART SHIELDS IN'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='CONTINENTAL ILLINOIS','acquiror'] = 'CONTINENTAL ILLINOIS NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='LF ROTHSCHILD HOLDINGS','acquiror'] = 'LF ROTHSCHILD'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='BANK OF BOSTON CORPBOSTONMA','acquiror'] = 'BANK OF BOSTON'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='COLONIAL BANCGROUP','acquiror'] = 'COLONIAL BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='PONTIAC STATE BANK PONTIAC','acquiror'] = 'PONTIAC STATE BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='COMMERCIAL BANK & TR','acquiror'] = 'COMMERCIAL BANK & TRUST'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST CAPITAL','acquiror'] = 'FIRST CAPITAL BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='HERITAGE BANK NJ','acquiror'] = 'HERITAGE BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='SECURITY NATIONAL','acquiror'] = 'SECURITY NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST NATIONAL STATE BANK','acquiror'] = 'FIRST NATIONAL STATE BANK NJ'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='MELLON NATIONAL','acquiror'] = 'MELLON NATIONAL BANK & TR'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='UNITED JERSEY BANKSPRINCETONNJ','acquiror'] = 'UNITED JERSEY BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='BANC ONE CORPCOLUMBUSOHIO','acquiror'] = 'BANC ONE'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='BANK OF NEW ENGLANDBOSTONMA','acquiror'] = 'BANK OF NEW ENGLAND'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='AMERITRUST CORPCLEVELANDOH','acquiror'] = 'AMERITRUST'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='CENTRAL FIDELITY BANKS INCVA','acquiror'] = 'CENTRAL FIDELITY BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='WESTAMERICA BANKCALIFORNIA','acquiror'] = 'WESTAMERICA BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='NORTH FORK BANKMELVILLENY','acquiror'] = 'NORTH FORK BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='HUDSON UNITED BANKNJ','acquiror'] = 'HUDSON UNITED BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='DIME BANK INCNEW YORKNY','acquiror'] = 'DIME BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='MICHIGAN NATIONAL','acquiror'] = 'MICHIGAN NATIONAL BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='COMERICA','acquiror'] = 'COMERICA BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST OF MICHIGAN CAPITAL','acquiror'] = 'FIRST OF MICHIGAN'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='PIEDMONT','acquiror'] = 'PIEDMONT BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='NORTHEAST BANKME','acquiror'] = 'NORTHEAST BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='PROGRESSIVE SECURITIES','acquiror'] = 'PROGRESSIVE'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='MIDLANTIC BANKS INCEDISONNJ','acquiror'] = 'MIDLANTIC BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='TRUSTCO BANK CORP NY','acquiror'] = 'TRUSTCO BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='TROY FINANCIAL CORPTROYNY','acquiror'] = 'TROY FINANCIAL'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='KEY STATE BANK OWOSSO','acquiror'] = 'KEY STATE BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='QUICK & REILLY GROUP','acquiror'] = 'QUICK & REILLY'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST PENNSYLVANIA BANK','acquiror'] = 'FIRST PENNSYLVANIA'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='CENTERRE BANKST LOUIS','acquiror'] = 'CENTERRE BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST FIDELITY BANKNJ','acquiror'] = 'FIRST FIDELITY BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='HSBC HONG KONG','acquiror'] = 'HSBC'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='PNC FINANCIALPITTSBURGHPA','acquiror'] = 'PNC BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='DELAWARE TRUST CODE','acquiror'] = 'DELAWARE TRUST'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='AMSOUTH BANKALABAMA','acquiror'] = 'AMSOUTH BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='EF HUTTON GROUP','acquiror'] = 'E F HUTTON'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='DEAN WITTER FINANCIAL SERVICES','acquiror'] = 'DEAN WITTER'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='DIME BANKMARIETTAOHIO','acquiror'] = 'DIME BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='NATIONAL BANK OF COMMERCEDC','acquiror'] = 'NATIONAL BANK OF COMMERCE'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='A BROWN & SONS','acquiror'] = 'ALEX BROWN & SONS'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='KEMPER','acquiror'] = 'KEMPER SECURITIES'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='CONNECTICUT BANK&TRHARTFORDCT','acquiror'] = 'CONNECTICUT BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='TORONTO DOMINION BANK','acquiror'] = 'TD BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST HORIZON','acquiror'] = 'FIRST HORIZON BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='KIDDER PEABODY GROUP(GEN ELEC)','acquiror'] = 'KIDDER PEABODY'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='MID AMERICA NATIONAL BANK','acquiror'] = 'MID AMERICAN NATIONAL BANK & TR'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='GREAT SOUTHERN BANKLANTANAFL','acquiror'] = 'GREAT SOUTHERN BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='RODMAN & RENSHAW CAPITAL GROUP','acquiror'] = 'RODMAN & RENSHAW'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='STERLING BANK CO FSB','acquiror'] = 'STERLING BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='PROVIDENT BANK CORPMD','acquiror'] = 'PROVIDENT BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FRONTIER BANK NALA PALMACA','acquiror'] = 'FRONTIER BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='MARK TWAIN BANKMO','acquiror'] = 'MARK TWAIN BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='ST JAMES BANKLUTCHERLA','acquiror'] = 'ST JAMES BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='REGIONS FINANCIAL CORPAL','acquiror'] = 'REGIONS BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST COMMONWEALTH BANKWISEVA','acquiror'] = 'FIRST COMMONWEALTH BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='PREMIER BANKSHSBLUEFIELDVA','acquiror'] = 'PREMIER BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='BANK PLUS CORPLOS ANGELESCA','acquiror'] = 'BANK PLUS'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='CHASE MANHATTAN CORPNY','acquiror'] = 'CHASE MANHATTAN'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='NATIONAL CITY BANK INCIN','acquiror'] = 'NATIONAL CITY BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='CNB BANK INCIN','acquiror'] = 'CNB BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='HSBC HOLDINGS','acquiror'] = 'HSBC'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST SECURITY CORPUTAH','acquiror'] = 'FIRST SECURITY'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='STURGIS BANK&TRUSTTHREE','acquiror'] = 'STURGIS BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='SOUTHERN MICHIGAN BANKMI','acquiror'] = 'SOUTHERN MICHIGAN BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='GOLDMAN SACHS GROUP','acquiror'] = 'GOLDMAN SACHS'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='BSB BANK INCBINGHAMTONNY','acquiror'] = 'BSB BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST WESTERN BANKNC','acquiror'] = 'FIRST WESTERN BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='AG EDWARDS','acquiror'] = 'A G EDWARDS & SONS'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='LADENBURG THALMANN FINL SERVICES','acquiror'] = 'LADENBURG THALMANN'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='FIRST BANK OF MISSOURI BANK','acquiror'] = 'FIRST BANK OF MISSOURI'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='STERLING BANK HOUSTON TEXAS','acquiror'] = 'STERLING BANK'
MA_withdrawn.loc[MA_withdrawn['acquiror']=='TRI VALLEY BANKSAN RAMONCA','acquiror'] = 'TRI VALLEY BANK'


In [7]:
%%script false --no-raise-error

# Note that results in the merge below has been used to edit the prior cell

def proc_list(MA_withdrawn):

    fuzzy_matches_targets = []
    for idx,row in MA_withdrawn.iterrows():
        ratio = process.extract(row['target'],GPF_names['name_GPF'],limit=3,scorer=fuzz.partial_ratio)
        for item in ratio:
            if item[1]>=90:
                fuzzy_matches_targets = fuzzy_matches_targets+[{
                    'Target':row['target'],
                    'name_GPF':item[0],
                    'score':item[1],
                }]
    fuzzy_matches_targets = pd.DataFrame(fuzzy_matches_targets)
    
    return fuzzy_matches_targets

MA_withdrawn_dd = dd.from_pandas(MA_withdrawn, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    fuzzy_matches_targets = MA_withdrawn_dd.map_partitions(proc_list, 
    meta=pd.DataFrame(columns=['Target','name_GPF','score'])).compute()

def proc_list(MA_withdrawn):

    fuzzy_matches_acquirers = []
    for idx,row in MA_withdrawn.iterrows():
        ratio = process.extract(row['acquiror'],GPF_names['name_GPF'],limit=3,scorer=fuzz.partial_ratio)
        for item in ratio:
            if item[1]>=90:
                fuzzy_matches_acquirers = fuzzy_matches_acquirers+[{
                    'Buyer':row['acquiror'],
                    'name_GPF':item[0],
                    'score':item[1],
                }]
    fuzzy_matches_acquirers = pd.DataFrame(fuzzy_matches_acquirers)
    
    return fuzzy_matches_acquirers

MA_withdrawn_dd = dd.from_pandas(MA_withdrawn, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    fuzzy_matches_acquirers = MA_withdrawn_dd.map_partitions(proc_list, 
    meta=pd.DataFrame(columns=['Buyer','name_GPF','score'])).compute()

fuzzy_matches_acquirers = fuzzy_matches_acquirers.rename(columns={'Buyer':'acquiror','name_GPF':'name_GPF_acquiror'})
fuzzy_matches_targets = fuzzy_matches_targets.rename(columns={'Target':'target','name_GPF':'name_GPF_target'})
MA_withdrawn = MA_withdrawn[['target','acquiror','sale_year']].merge(
    fuzzy_matches_acquirers,on='acquiror').drop_duplicates().\
    drop(columns=['score'])
MA_withdrawn = MA_withdrawn[['target','acquiror','sale_year','name_GPF_acquiror']].merge(
    fuzzy_matches_targets,on='target').drop_duplicates().\
    drop(columns=['score'])


In [8]:
SNL_withdrawn = SNL_withdrawn[(SNL_withdrawn['Buyer'].isin(GPF_names['name_GPF']))&(SNL_withdrawn['Target'].isin(GPF_names['name_GPF']))]
SNL_withdrawn['announce_year'] = SNL_withdrawn['Announcement Date'].dt.year
SNL_withdrawn = SNL_withdrawn.rename(columns={'Buyer':'acquiror','Target':'target'})
SNL_withdrawn = SNL_withdrawn[['target','acquiror','announce_year']]

MA_withdrawn = MA_withdrawn[(MA_withdrawn['acquiror'].isin(GPF_names['name_GPF']))&(MA_withdrawn['target'].isin(GPF_names['name_GPF']))]
MA_withdrawn = MA_withdrawn[['target','acquiror','announce_year']]
MA_withdrawn = pd.concat([SNL_withdrawn,MA_withdrawn]).drop_duplicates().reset_index(drop=True).sort_values('announce_year')
MA_withdrawn = MA_withdrawn[MA_withdrawn['target']!=MA_withdrawn['acquiror']]

In [9]:
MA_withdrawn.to_csv("../CleanData/SDC/0I_MA_withdrawn.csv")

# 2. FDIC Deposit

Notes:
- The firm names have been cleaned so all M&As in https://en.wikipedia.org/wiki/List_of_bank_mergers_in_the_United_States from 1995 on will be in the sample.
- I also clean names of banks in https://en.wikipedia.org/wiki/List_of_largest_banks_in_the_United_States for both SOD and SNL, so they are consistent.

In [10]:
#------------#
# Import SOD #
#------------#

SOD = pd.DataFrame()
for year in range(1994,2024):
    SOD_oneyear = pd.read_csv('../RawData/FDIC/ALL_'+str(year)+'/ALL_'+str(year)+'.csv',encoding="ISO-8859-1",low_memory=False)
    SOD_oneyear = SOD_oneyear[['YEAR','NAMEFULL','ADDRESBR','BRNUM','NAMEBR','CITYBR','CNTYNAMB','STALPBR','ZIPBR','DEPSUMBR']]
    SOD = pd.concat([SOD,SOD_oneyear])

SOD = SOD.rename(columns={
    'YEAR':'year',
    'NAMEFULL':'name',
    'CITYBR':'City',
    'CNTYNAMB':'County',
    'STALPBR':'State',
    })

SOD['County'] = SOD['County'].str.upper()
SOD['County'] = SOD['County'].str.replace(' COUNTY','')
SOD['County'] = SOD['County'].str.replace(' PARISH','')
SOD['County'] = SOD['County'].str.replace(' AND ',' & ')
SOD['County'] = SOD['County'].str.replace('.','',regex=False)

#-------------#
# Import CBSA #
#-------------#

%run -i SCRIPT_us_states.py

# "CSA" is for metropolitan and "CBSA" includes also those micropolitan
CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]

# Add state abbreviations
us_state_to_abbrev = pd.DataFrame.from_dict(us_state_to_abbrev,orient='index').reset_index()
us_state_to_abbrev.columns = ['State Name','State']
CBSAData = CBSAData.rename(columns={'County/County Equivalent':'County'})
CBSAData = CBSAData.merge(us_state_to_abbrev,on='State Name',how='outer',indicator=True)
CBSAData = CBSAData[CBSAData['_merge']=='both'].drop(columns=['_merge'])
# Merge is perfect
CBSAData['County'] = CBSAData['County'].str.upper()
CBSAData['County'] = CBSAData['County'].str.replace(' COUNTY','')
CBSAData['County'] = CBSAData['County'].str.replace(' PARISH','')
CBSAData['County'] = CBSAData['County'].str.replace(' AND ',' & ')
CBSAData['County'] = CBSAData['County'].str.replace('.','',regex=False)
CBSAData['CSA Code'] = CBSAData['CSA Code'].astype(float)
CBSAData['CBSA Code'] = CBSAData['CBSA Code'].astype(float)

CBSAData = CBSAData[['County','State','CSA Code','CSA Title','CBSA Code','CBSA Title']]

SOD = SOD.merge(CBSAData,on=['County','State'])

SOD['name'] = SOD['name'].apply(FUN_proc_name)
SOD['name'] = SOD['name'].str.replace('*','',regex=False)
SOD['name'] = SOD['name'].apply(lambda x: x[len("THE "):] if x.startswith("THE ") else x)

In [11]:
#------------------------------------------------------------#
# Clean SOD names and make it consistent with records in SNL #
#------------------------------------------------------------#

SOD.loc[SOD['name']=='BBVA COMPASS','name'] = 'BBVA USA BANK'
SOD.loc[SOD['name']=='PNC BANK FSB','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC BANK NEW ENGLAND','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC BANK DELAWARE','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC NATIONAL BANK','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC NATIONAL BANK OF DELAWARE','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC BANK NORTHERN KENTUCKY','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC MORTGAGE BANK','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC BANK NEW JERSEY','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='PNC BANK NEW JERSEY','name'] = 'PNC BANK'
SOD.loc[SOD['name']=='CANADIAN IMPERIAL BANK OF COMMERCE (NEW YORK)','name'] = 'CIBC HOLDCO'
SOD.loc[SOD['name']=='CIBC','name'] = 'CIBC HOLDCO'
SOD.loc[SOD['name']=='FIRST NIAGARA BANK','name'] = 'FIRST NIAGARA FINANCIAL GROUP'
SOD.loc[SOD['name']=='FIRST NIAGARA COMMERCIAL BANK','name'] = 'FIRST NIAGARA FINANCIAL GROUP'
SOD.loc[SOD['name']=='FIRST NATIONAL BANK OF NIAGARA','name'] = 'FIRST NIAGARA FINANCIAL GROUP'
SOD.loc[SOD['name']=='KEY BANK','name'] = 'KEYCORP'
SOD.loc[SOD['name']=='KEY BANK OF FLORIDA','name'] = 'KEYCORP'
SOD.loc[SOD['name']=='KEY BANK USA','name'] = 'KEYCORP'
SOD.loc[SOD['name']=='KEY BISCAYNE BANK','name'] = 'KEYCORP'
SOD.loc[SOD['name']=='KEY BANK OF VERMONT','name'] = 'KEYCORP'
SOD.loc[SOD['name']=='HUNTINGTON NATIONAL BANK','name'] = 'HUNTINGTON BANK'
SOD.loc[SOD['name']=='HUNTINGTON FEDERAL SAVINGS BANK','name'] = 'HUNTINGTON BANK'
SOD.loc[SOD['name']=='HUDSON CITY SAVINGS BANK','name'] = 'HUDSON CITY BANK'
SOD.loc[SOD['name']=='HUDSON CITY SAVINGS INSTITUTION','name'] = 'HUDSON CITY BANK'
SOD.loc[SOD['name']=='ALLIANCE BANK','name'] = 'ALLIANCE FINANCIAL'
SOD.loc[SOD['name']=='CAPITAL ONE','name'] = 'CAPITAL ONE FINANCIAL'
SOD.loc[SOD['name']=='CAPITAL ONE BANK (USA)','name'] = 'CAPITAL ONE FINANCIAL'
SOD.loc[SOD['name']=='CAPITAL ONE FSB','name'] = 'CAPITAL ONE FINANCIAL'
SOD.loc[SOD['name']=='WILMINGTON TRUST FSB','name'] = 'WILMINGTON TRUST'
SOD.loc[SOD['name']=='WILMINGTON TRUST OF PENNSYLVANIA','name'] = 'WILMINGTON TRUST'
SOD.loc[SOD['name']=='BRADFORD FEDERAL SAVINGS BANK','name'] = 'BRADFORD BANK'
SOD.loc[SOD['name']=='TD BANK USA','name'] = 'TD BANK'
SOD.loc[SOD['name']=='TD BANK USA FSB','name'] = 'TD BANK'
SOD.loc[SOD['name']=='PARTNERS TRUST BANK','name'] = 'PARTNERS TRUST FINANCIAL GROUP'
SOD.loc[SOD['name']=='PARTNERS TRUST MUNICIPAL BANK','name'] = 'PARTNERS TRUST FINANCIAL GROUP'
SOD.loc[SOD['name']=='MELLON BANK','name'] = 'MELLON FINANCIAL'
SOD.loc[SOD['name']=='BANCO BILBAO VIZCAYA','name'] = 'BBVA BANK'
SOD.loc[SOD['name']=='BBVA USA BANK','name'] = 'BBVA BANK'
SOD.loc[SOD['name']=='BBVA USA','name'] = 'BBVA BANK'
SOD.loc[SOD['name']=='BBVA BANCOMER USA','name'] = 'BBVA BANK'
SOD.loc[SOD['name']=='COMPASS BANK FOR SAVINGS','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS BANK HOUSTON','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS BANK FOR SAVINGS','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS BANK SAN ANTONIO','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS COMMUNITY BANK','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS BANK CENTRAL TEXAS','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS FEDERAL SAVINGS & LOAN ASSOCIATION','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS FEDERAL SAVINGS BANK','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='COMPASS SAVINGS BANK','name'] = 'COMPASS BANK'
SOD.loc[SOD['name']=='WESTERN FINANCIAL SAVINGS BANK FSB','name'] = 'WESTCORP'
SOD.loc[SOD['name']=='WESTERN FINANCIAL BANK FSB','name'] = 'WESTCORP'
SOD.loc[SOD['name']=='WESTERN FINANCIAL BANK','name'] = 'WESTCORP'
SOD.loc[SOD['name']=='HIBERNIA SAVINGS BANK','name'] = 'HIBERNIA BANK'
SOD.loc[SOD['name']=='HIBERNIA NATIONAL BANK','name'] = 'HIBERNIA BANK'
SOD.loc[SOD['name']=='HIBERNIA NATIONAL BANK OF TEXAS','name'] = 'HIBERNIA BANK'
SOD.loc[SOD['name']=='RIGGS NATIONAL BANK OF VIRGINIA','name'] = 'RIGGS NATIONAL'
SOD.loc[SOD['name']=='RIGGS BANK','name'] = 'RIGGS NATIONAL'
SOD.loc[SOD['name']=='RIGGS NATIONAL BANK OF WASHINGTON D C','name'] = 'RIGGS NATIONAL'
SOD.loc[SOD['name']=='RIGGS NATIONAL BANK OF MARYLAND','name'] = 'RIGGS NATIONAL'
SOD.loc[SOD['name']=='JPMORGAN CHASE BANK','name'] = 'JPMORGAN CHASE'
SOD.loc[SOD['name']=='BANK OF AMERICA NATIONAL TRUST & SAVINGS ASSOCIATION','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA GEORGIA','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA FEDERAL SAVINGS BANK','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA NA (USA)','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA NEW MEXICO','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA CALIFORNIA','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA COMMUNITY DEVELOPMENT BANK','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA IDAHO','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA RHODE ISLAND NATIONAL ASSOCATION','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA RHODE ISLAND','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='BANK OF AMERICA ARIZONA','name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name']=='TRUST CO OF NEW JERSEY','name'] = 'TRUST CO OF NEW JERSEY (THE)'
SOD.loc[SOD['name']=='ALLFIRST BANK','name'] = 'ALLFIRST FINANCIAL'
SOD.loc[SOD['name']=='US BANK NA ND','name'] = 'US BANK'
SOD.loc[SOD['name']=='US BANK OF IDAHO','name'] = 'US BANK'
SOD.loc[SOD['name']=='US BANK NA OR','name'] = 'US BANK'
SOD.loc[SOD['name']=='US BANK OF UTAH','name'] = 'US BANK'
SOD.loc[SOD['name']=='US BANK OF SOUTHWEST WASHINGTON','name'] = 'US BANK'
SOD.loc[SOD['name']=='US BANK NA MT','name'] = 'US BANK'
SOD.loc[SOD['name']=='REPUBLIC NATIONAL BANK OF NEW YORK','name'] = 'REPUBLIC NEW YORK'
SOD.loc[SOD['name']=='HOME SAVINGS OF AMERICA FSB','name'] = 'AHMANSON & CO (HF)'
SOD.loc[SOD['name']=='HOME SAVINGS OF AMERICA','name'] = 'AHMANSON & CO (HF)'
SOD.loc[SOD['name']=='STAR BANK NA KENTUCKY','name'] = 'STAR BANK'
SOD.loc[SOD['name']=='BANK OF BOSTON (MAINE)','name'] = 'BANK OF BOSTON'
SOD.loc[SOD['name']=='MUFG UNION BANK','name'] = 'UNION BANK'
SOD.loc[SOD['name']=='SOUTHERN NATIONAL BANK OF SOUTH CAROLINA','name'] = 'SOUTHERN NATIONAL BANK'
SOD.loc[SOD['name']=='SOUTHERN NATIONAL BANK OF TEXAS','name'] = 'SOUTHERN NATIONAL BANK'
SOD.loc[SOD['name']=='SOUTHERN NATIONAL BANK OF NORTH CAROLINA','name'] = 'SOUTHERN NATIONAL BANK'
SOD.loc[SOD['name']=='UMB BANK BOONVILLE','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK ARIZONA','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK JEFFERSON CITY','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK OMAHA','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK NORTHEAST','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK NORTHWEST','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK CASS COUNTY','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK KANSAS','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK WARRENSBURG','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK SOUTHWEST','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='UMB BANK OF ST LOUIS','name'] = 'UMB BANK'
SOD.loc[SOD['name']=='PACIFIC WESTERN NATIONAL BANK','name'] = 'PACIFIC WESTERN BANK'
SOD.loc[SOD['name']=='HANCOCK BANK OF ALABAMA','name'] = 'HANCOCK WHITNEY'
SOD.loc[SOD['name']=='HANCOCK BANK OF FLORIDA','name'] = 'HANCOCK WHITNEY'
SOD.loc[SOD['name']=='HANCOCK BANK','name'] = 'HANCOCK WHITNEY'
SOD.loc[SOD['name']=='WILMINGTON SAVINGS FUND SOCIETY FSB','name'] = 'WSFS FINANCIAL'

SOD.loc[SOD['name'].str.contains('SUNTRUST BANK'),'name'] = 'SUNTRUST BANKS'
SOD.loc[SOD['name'].str.contains('PROVIDENT BANK'),'name'] = 'PROVIDENT BANK'
SOD.loc[SOD['name'].str.contains('NATIONAL CITY BANK'),'name'] = 'NATIONAL CITY CORP'
SOD.loc[SOD['name'].str.contains('FIFTH THIRD BANK'),'name'] = 'FIFTH THIRD BANK'
SOD.loc[SOD['name'].str.contains('WASHINGTON MUTUAL'),'name'] = 'WASHINGTON MUTUAL BANK'
SOD.loc[SOD['name'].str.contains('WELLS FARGO'),'name'] = 'WELLS FARGO'
SOD.loc[SOD['name'].str.contains('WACHOVIA'),'name'] = 'WACHOVIA'
SOD.loc[SOD['name'].str.contains('MERRILL LYNCH'),'name'] = 'MERRILL LYNCH'
SOD.loc[SOD['name'].str.contains('WORLD SAVINGS'),'name'] = 'GOLDEN WEST FINANCIAL'
SOD.loc[SOD['name'].str.contains('LA SALLE BANK'),'name'] = 'LA SALLE BANK'
SOD.loc[SOD['name'].str.contains('LASALLE BANK'),'name'] = 'LA SALLE BANK'
SOD.loc[SOD['name'].str.contains('AMSOUTH BANK'),'name'] = 'AMSOUTH BANK'
SOD.loc[SOD['name'].str.contains('REGIONS BANK'),'name'] = 'REGIONS BANK'
SOD.loc[SOD['name'].str.contains('CORNERSTONE BANK'),'name'] = 'CORNERSTONE BANK'
SOD.loc[SOD['name'].str.contains('MBNA'),'name'] = 'MBNA'
SOD.loc[SOD['name'].str.contains('UNION PLANTERS'),'name'] = 'UNION PLANTERS'
SOD.loc[SOD['name'].str.contains('POPULAR'),'name'] = 'POPULAR'
SOD.loc[SOD['name'].str.contains('BANK ONE'),'name'] = 'BANC ONE'
SOD.loc[SOD['name'].str.contains('FLEET'),'name'] = 'FLEET BANK'
SOD.loc[SOD['name'].str.contains('FIRST VIRGINIA BANK'),'name'] = 'FIRST VIRGINIA BANKS'
SOD.loc[SOD['name'].str.contains('WASHINGTON MUTUAL'),'name'] = 'WASHINGTON MUTUAL'
SOD.loc[SOD['name'].str.contains('STANDARD FEDERAL'),'name'] = 'STANDARD FEDERAL BANK'
SOD.loc[SOD['name'].str.contains('OLD KENT'),'name'] = 'OLD KENT FINANCIAL'
SOD.loc[SOD['name'].str.contains('FIRST UNION'),'name'] = 'FIRST UNION'
SOD.loc[SOD['name'].str.contains('FIRSTAR'),'name'] = 'FIRSTAR'
SOD.loc[SOD['name'].str.contains('FIRST SECURITY'),'name'] = 'FIRST SECURITY'
SOD.loc[SOD['name'].str.contains('WELLS FARGO'),'name'] = 'WELLS FARGO'
SOD.loc[SOD['name'].str.contains('CHASE BANK'),'name'] = 'CHASE BANK'
SOD.loc[SOD['name'].str.contains('JPMORGAN CHASE'),'name'] = 'CHASE BANK'
SOD.loc[SOD['name'].str.contains('CHASE MANHATTAN'),'name'] = 'CHASE BANK'
SOD.loc[SOD['name'].str.contains('FIRST AMERICAN'),'name'] = 'FIRST AMERICAN NATIONAL BANK'
SOD.loc[SOD['name'].str.contains('MERCANTILE BANK'),'name'] = 'MERCANTILE BANK'
SOD.loc[SOD['name'].str.contains('HSBC'),'name'] = 'HSBC'
SOD.loc[SOD['name'].str.contains('DEUTSCHE BANK'),'name'] = 'DEUTSCHE BANK AG'
SOD.loc[SOD['name'].str.contains('BANKERS TRUST'),'name'] = 'BANKERS TRUST'
SOD.loc[SOD['name'].str.contains('BANKBOSTON'),'name'] = 'BANKBOSTON'
SOD.loc[SOD['name'].str.contains('CRESTAR'),'name'] = 'CRESTAR FINANCIAL'
SOD.loc[SOD['name'].str.contains('FIRST COMMERCE'),'name'] = 'FIRST COMMERCE'
SOD.loc[SOD['name'].str.contains('BANC ONE'),'name'] = 'BANC ONE'
SOD.loc[SOD['name'].str.contains('NORWEST'),'name'] = 'NORWEST'
SOD.loc[SOD['name'].str.contains('FIRST NATIONWIDE'),'name'] = 'FIRST NATIONWIDE BANK'
SOD.loc[SOD['name'].str.contains('BANK OF AMERICA'),'name'] = 'BANK OF AMERICA'
SOD.loc[SOD['name'].str.contains('BANKAMERICA'),'name'] = 'BANKAMERICA'
SOD.loc[SOD['name'].str.contains('NATIONSBANK'),'name'] = 'NATIONSBANK'
SOD.loc[SOD['name'].str.contains('CORESTATES'),'name'] = 'CORESTATES FINANCIAL'
SOD.loc[SOD['name'].str.contains('FIRST UNION'),'name'] = 'FIRST UNION'
SOD.loc[SOD['name'].str.contains('BARNETT BANK'),'name'] = 'BARNETT BANKS'
SOD.loc[SOD['name'].str.contains('CALIFORNIA FEDERAL BANK'),'name'] = 'CAL FED BANK'
SOD.loc[SOD['name'].str.contains('NATIONAL CITY'),'name'] = 'NATIONAL CITY'
SOD.loc[SOD['name'].str.contains('FIRST OF AMERICA BANK'),'name'] = 'FIRST OF AMERICA BANK'
SOD.loc[SOD['name'].str.contains('SIGNET BANK'),'name'] = 'SIGNET BANK'
SOD.loc[SOD['name'].str.contains('GREAT WESTERN'),'name'] = 'GREAT WESTERN FINANCIAL'
SOD.loc[SOD['name'].str.contains('BOATMENS'),'name'] = 'BOATMENS BANK'
SOD.loc[SOD['name'].str.contains('NATIONAL WESTMINSTER BANK'),'name'] = 'NATIONAL WESTMINSTER BANK'
SOD.loc[SOD['name'].str.contains('NATWEST'),'name'] = 'NATWEST'
SOD.loc[SOD['name'].str.contains('CENTERBANK'),'name'] = 'CENTER FINANCIAL'
SOD.loc[SOD['name'].str.contains('FIRST FIDELITY'),'name'] = 'FIRST FIDELITY BANK'
SOD.loc[SOD['name'].str.contains('BAYBANK'),'name'] = 'BAYBANKS'
SOD.loc[SOD['name'].str.contains('FIRST INTERSTATE BANK'),'name'] = 'FIRST INTERSTATE BANK'
SOD.loc[SOD['name'].str.contains('SHAWMUT BANK'),'name'] = 'SHAWMUT BANK'
SOD.loc[SOD['name'].str.contains('BB&T'),'name'] = 'BB&T'
SOD.loc[SOD['name'].str.contains('NATIONAL BANK OF DETROIT'),'name'] = 'NBD'
SOD.loc[SOD['name'].str.contains('NBD'),'name'] = 'NBD'
SOD.loc[SOD['name'].str.contains('GOLDMAN SACHS'),'name'] = 'GOLDMAN SACHS'
SOD.loc[SOD['name'].str.contains('MORGAN STANLEY'),'name'] = 'MORGAN STANLEY'
SOD.loc[SOD['name'].str.contains('CHARLES SCHWAB'),'name'] = 'CHARLES SCHWAB'
SOD.loc[SOD['name'].str.contains('BANK OF NEW YORK MELLON'),'name'] = 'BANK OF NEW YORK MELLON'
SOD.loc[SOD['name'].str.contains('STATE STREET'),'name'] = 'STATE STREET'
SOD.loc[SOD['name'].str.contains('BMO'),'name'] = 'BMO BANK'
SOD.loc[SOD['name'].str.contains('AMERICAN EXPRESS'),'name'] = 'AMERICAN EXPRESS'
SOD.loc[SOD['name'].str.contains('FIRST CITIZENS'),'name'] = 'FIRST CITIZENS'
SOD.loc[SOD['name'].str.contains('M&T'),'name'] = 'M&T BANK'
SOD.loc[SOD['name'].str.contains('FIFTH THIRD'),'name'] = 'FIFTH THIRD BANK'
SOD.loc[SOD['name'].str.contains('UBS BANK USA'),'name'] = 'UBS AG'
SOD.loc[SOD['name'].str.contains('KEYBANK'),'name'] = 'KEYCORP'
SOD.loc[SOD['name'].str.contains('KEY BANK'),'name'] = 'KEYCORP'
SOD.loc[SOD['name'].str.contains('KEY CORP'),'name'] = 'KEYCORP'
SOD.loc[SOD['name'].str.contains('BARCLAYS'),'name'] = 'BARCLAYS'
SOD.loc[SOD['name'].str.contains('HUNTINGTON BANK'),'name'] = 'HUNTINGTON BANK'
SOD.loc[SOD['name'].str.contains('HUNTINGTON NATIONAL BANK'),'name'] = 'HUNTINGTON BANK'
SOD.loc[SOD['name'].str.contains('SANTANDER'),'name'] = 'SANTANDER BANK'
SOD.loc[SOD['name'].str.contains('AMERIPRISE'),'name'] = 'AMERIPRISE FINANCIAL'
SOD.loc[SOD['name'].str.contains('RBC'),'name'] = 'RBC BANK'
SOD.loc[SOD['name'].str.contains('NORTHERN TRUST'),'name'] = 'NORTHERN TRUST'
SOD.loc[SOD['name'].str.contains('REGIONS BANK'),'name'] = 'REGIONS BANK'
SOD.loc[SOD['name'].str.contains('COMERICA'),'name'] = 'COMERICA BANK'
SOD.loc[SOD['name'].str.contains('MIZUHO'),'name'] = 'MIZUHO BANK'
SOD.loc[SOD['name'].str.contains('RAYMOND JAMES'),'name'] = 'RAYMOND JAMES'
SOD.loc[SOD['name'].str.contains('EAST WEST'),'name'] = 'EAST WEST BANK'
SOD.loc[SOD['name'].str.contains('CIBC'),'name'] = 'CIBC'
SOD.loc[SOD['name'].str.contains('SYNOVUS'),'name'] = 'SYNOVUS BANK'
SOD.loc[SOD['name'].str.contains('FROST NATIONAL BANK'),'name'] = 'CULLENFROST BANK'
SOD.loc[SOD['name'].str.contains('FROST BANK'),'name'] = 'CULLENFROST BANK'
SOD.loc[SOD['name'].str.contains('ASSOCIATED BANK'),'name'] = 'ASSOCIATED BANC'
SOD.loc[SOD['name'].str.contains('STIFEL'),'name'] = 'STIFEL'
SOD.loc[SOD['name'].str.contains('MIDFIRST'),'name'] = 'MIDFIRST BANK'
SOD.loc[SOD['name'].str.contains('TIAA'),'name'] = 'TIAA'
SOD.loc[SOD['name'].str.contains('BANKUNITED'),'name'] = 'BANKUNITED'
SOD.loc[SOD['name'].str.contains('BEAL BANC'),'name'] = 'BEAL BANK'
SOD.loc[SOD['name'].str.contains('BEAL BANK'),'name'] = 'BEAL BANK'
SOD.loc[SOD['name'].str.contains('FIRSTBANK'),'name'] = 'FIRSTBANK'
SOD.loc[SOD['name'].str.contains('SIMMONS'),'name'] = 'SIMMONS BANK'
SOD.loc[SOD['name'].str.contains('GLACIER'),'name'] = 'GLACIER BANK'
SOD.loc[SOD['name'].str.contains('FULTON BANK'),'name'] = 'FULTON BANK'
SOD.loc[SOD['name'].str.contains('FULTON SAVING'),'name'] = 'FULTON BANK'
SOD.loc[SOD['name'].str.contains('UNITED COMMUNITY'),'name'] = 'UNITED COMMUNITY BANK'
SOD.loc[SOD['name'].str.contains('FIRST HAWAIIAN'),'name'] = 'FIRST HAWAIIAN BANK'
SOD.loc[SOD['name'].str.contains('WASHINGTON FEDERAL'),'name'] = 'WASHINGTON FEDERAL'

SOD['name'] = SOD['name'].apply(lambda x: 'CITIZENS BANK' if x.startswith('CITIZENS BANK') else x)

# A list of banks in SOD
banks_in_SOD = SOD[['name']].drop_duplicates().reset_index(drop=True)


In [12]:
#-----------------#
# Clean SNL names #
#-----------------#

# While I make SOD names consistent with SNL, sometimes name within SNL is not consistent

SNL.loc[SNL['Target']=='FIRSTMERIT','Target'] = 'FIRSTMERIT BANK'
SNL.loc[SNL['Target']=='CAPITAL ONE','Target'] = 'CAPITAL ONE FINANCIAL'
SNL.loc[SNL['Target']=='NATIONAL CITY CORP','Target'] = 'NATIONAL CITY CORP'
SNL.loc[SNL['Target']=='FIRST CHARTER CORP','Target'] = 'FIRST CHARTER BANK'
SNL.loc[SNL['Target']=='TORONTO DOMINION BANK','Target'] = 'TD BANK'
SNL.loc[SNL['Target']=='THE TORONTO DOMINION BANK','Target'] = 'TD BANK'
SNL.loc[SNL['Target']=='HIBERNIA','Target'] = 'HIBERNIA BANK'
SNL.loc[SNL['Target']=='PNC FINANCIAL SERVICES GROUP INC (THE)','Target'] = 'PNC BANK'
SNL.loc[SNL['Target']=='FLEET FINANCIAL GROUP','Target'] = 'FLEET BANK'
SNL.loc[SNL['Target']=='FLEET NATIONAL BANK','Target'] = 'FLEET BANK'
SNL.loc[SNL['Target']=='FLEET BOSTON','Target'] = 'FLEET BANK'
SNL.loc[SNL['Target']=='JP MORGAN CHASE','Target'] = 'CHASE BANK'
SNL.loc[SNL['Target']=='FIRST CHICAGO NBD','Target'] = 'FIRST CHICAGO BANK'
SNL.loc[SNL['Target']=='ALLY BANK','Target'] = 'ALLY FINANCIAL'
SNL.loc[SNL['Target']=='UBS AMERICAS','Target'] = 'UBS AG'
SNL.loc[SNL['Target']=='UBS GROUP AG','Target'] = 'UBS AG'
SNL.loc[SNL['Target']=='SYNCHRONY FINANCIAL','Target'] = 'SYNCHRONY BANK'
SNL.loc[SNL['Target']=='WEBSTER FINANCIAL','Target'] = 'WEBSTER BANK'
SNL.loc[SNL['Target']=='CULLENFROST BANKERS','Target'] = 'CULLENFROST BANK'
SNL.loc[SNL['Target']=='FNB SOUTHEAST','Target'] = 'FNB BANK'
SNL.loc[SNL['Target']=='FNB NEW MEXICO','Target'] = 'FNB BANK'
SNL.loc[SNL['Target']=='FNB FINANCIAL GROUP','Target'] = 'FNB BANK'
SNL.loc[SNL['Target']=='FNB FINANCIAL SERVICES','Target'] = 'FNB BANK'
SNL.loc[SNL['Target']=='SOUTH STATE','Target'] = 'SOUTH STATE BANK'
SNL.loc[SNL['Target']=='UMB FINANCIAL','Target'] = 'UMB BANK'
SNL.loc[SNL['Target']=='ASSOCIATED BANK GREEN BAY','Target'] = 'ASSOCIATED BANC'
SNL.loc[SNL['Target']=='HANCOCK BANK OF FLORIDA','Target'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Target']=='HANCOCK HOLDING','Target'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Target']=='HANCOCK BANK','Target'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Target']=='HANCOCK BANK MS','Target'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Target']=='HANCOCK BANK GULFPORT','Target'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Target']=='FIRST NATIONAL OF NEBRASKA','Target'] = 'FIRST NATIONAL BANK OF NEBRASKA'
SNL.loc[SNL['Target']=='FULTON FINANCIAL','Target'] = 'FULTON BANK'
SNL.loc[SNL['Target']=='ARVEST BANK GROUP','Target'] = 'ARVEST BANK'
SNL.loc[SNL['Target']=='HEARTLAND FINANCIAL USA','Target'] = 'HEARTLAND BANK'

SNL.loc[SNL['Target'].str.contains('WASHINGTON MUTUAL'),'Target'] = 'WASHINGTON MUTUAL'
SNL.loc[SNL['Target'].str.contains('ABN AMRO'),'Target'] = 'ABN AMRO'
SNL.loc[SNL['Target'].str.contains('HSBC'),'Target'] = 'HSBC'
SNL.loc[SNL['Target'].str.contains('BANC ONE'),'Target'] = 'BANC ONE'
SNL.loc[SNL['Target'].str.contains('FIRSTAR'),'Target'] = 'FIRSTAR'
SNL.loc[SNL['Target'].str.contains('FIRST NATIONWIDE'),'Target'] = 'FIRST NATIONWIDE BANK'
SNL.loc[SNL['Target'].str.contains('FIRST OF AMERICA BANK'),'Target'] = 'FIRST OF AMERICA BANK'
SNL.loc[SNL['Target'].str.contains('SIGNET BANK'),'Target'] = 'SIGNET BANK'
SNL.loc[SNL['Target'].str.contains('GREAT WESTERN'),'Target'] = 'GREAT WESTERN FINANCIAL'
SNL.loc[SNL['Target'].str.contains('CRESTAR'),'Target'] = 'CRESTAR FINANCIAL'
SNL.loc[SNL['Target'].str.contains('FIRST UNION'),'Target'] = 'FIRST UNION'
SNL.loc[SNL['Target'].str.contains('BB&T'),'Target'] = 'BB&T'
SNL.loc[SNL['Target'].str.contains('NBD'),'Target'] = 'NBD'
SNL.loc[SNL['Target'].str.contains('GOLDMAN SACHS'),'Target'] = 'GOLDMAN SACHS'
SNL.loc[SNL['Target'].str.contains('MORGAN STANLEY'),'Target'] = 'MORGAN STANLEY'
SNL.loc[SNL['Target'].str.contains('CHARLES SCHWAB'),'Target'] = 'CHARLES SCHWAB'
SNL.loc[SNL['Target'].str.contains('CAPITAL ONE'),'Target'] = 'CAPITAL ONE FINANCIAL'
SNL.loc[SNL['Target'].str.contains('BANK OF NEW YORK MELLON'),'Target'] = 'BANK OF NEW YORK MELLON'
SNL.loc[SNL['Target'].str.contains('STATE STREET'),'Target'] = 'STATE STREET'
SNL.loc[SNL['Target'].str.contains('BMO'),'Target'] = 'BMO BANK'
SNL.loc[SNL['Target'].str.contains('AMERICAN EXPRESS'),'Target'] = 'AMERICAN EXPRESS'
SNL.loc[SNL['Target'].str.contains('CITIZENS FINANCIAL'),'Target'] = 'CITIZENS BANK'
SNL.loc[SNL['Target'].str.contains('FIRST CITIZENS'),'Target'] = 'FIRST CITIZENS'
SNL.loc[SNL['Target'].str.contains('M&T'),'Target'] = 'M&T BANK'
SNL.loc[SNL['Target'].str.contains('FIFTH THIRD'),'Target'] = 'FIFTH THIRD BANK'
SNL.loc[SNL['Target'].str.contains('KEYBANK'),'Target'] = 'KEYCORP'
SNL.loc[SNL['Target'].str.contains('KEY BANK'),'Target'] = 'KEYCORP'
SNL.loc[SNL['Target'].str.contains('KEY CORP'),'Target'] = 'KEYCORP'
SNL.loc[SNL['Target'].str.contains('BARCLAYS'),'Target'] = 'BARCLAYS'
SNL.loc[SNL['Target'].str.contains('HUNTINGTON BANK'),'Target'] = 'HUNTINGTON BANK'
SNL.loc[SNL['Target'].str.contains('HUNTINGTON NATIONAL BANK'),'Target'] = 'HUNTINGTON BANK'
SNL.loc[SNL['Target'].str.contains('SANTANDER BANK'),'Target'] = 'SANTANDER BANK'
SNL.loc[SNL['Target'].str.contains('AMERIPRISE'),'Target'] = 'AMERIPRISE FINANCIAL'
SNL.loc[SNL['Target'].str.contains('RBC'),'Target'] = 'RBC BANK'
SNL.loc[SNL['Target'].str.contains('NORTHERN TRUST'),'Target'] = 'NORTHERN TRUST'
SNL.loc[SNL['Target'].str.contains('REGIONS BANK'),'Target'] = 'REGIONS BANK'
SNL.loc[SNL['Target'].str.contains('DEUTSCHE BANK'),'Target'] = 'DEUTSCHE BANK AG'
SNL.loc[SNL['Target'].str.contains('COMERICA'),'Target'] = 'COMERICA BANK'
SNL.loc[SNL['Target'].str.contains('FIRST HORIZON'),'Target'] = 'FIRST HORIZON BANK'
SNL.loc[SNL['Target'].str.contains('MIZUHO'),'Target'] = 'MIZUHO BANK'
SNL.loc[SNL['Target'].str.contains('RAYMOND JAMES'),'Target'] = 'RAYMOND JAMES'
SNL.loc[SNL['Target'].str.contains('EAST WEST'),'Target'] = 'EAST WEST BANK'
SNL.loc[SNL['Target'].str.contains('CIBC'),'Target'] = 'CIBC'
SNL.loc[SNL['Target'].str.contains('WINTRUST'),'Target'] = 'WINTRUST FINANCIAL'
SNL.loc[SNL['Target'].str.contains('PACIFIC WESTERN'),'Target'] = 'PACIFIC WESTERN BANK'
SNL.loc[SNL['Target'].str.contains('PACWEST'),'Target'] = 'PACIFIC WESTERN BANK'
SNL.loc[SNL['Target'].str.contains('STIFEL'),'Target'] = 'STIFEL'
SNL.loc[SNL['Target'].str.contains('MIDFIRST'),'Target'] = 'MIDFIRST BANK'
SNL.loc[SNL['Target'].str.contains('TIAA'),'Target'] = 'TIAA'
SNL.loc[SNL['Target'].str.contains('BANKUNITED'),'Target'] = 'BANKUNITED'
SNL.loc[SNL['Target'].str.contains('BEAL BANK'),'Target'] = 'BEAL BANK'
SNL.loc[SNL['Target'].str.contains('FIRST INTERSTATE'),'Target'] = 'FIRST INTERSTATE BANK'
SNL.loc[SNL['Target'].str.contains('SIMMONS'),'Target'] = 'SIMMONS BANK'
SNL.loc[SNL['Target'].str.contains('UNITED COMMUNITY'),'Target'] = 'UNITED COMMUNITY BANK'
SNL.loc[SNL['Target'].str.contains('SOFI'),'Target'] = 'SOFI BANK'
SNL.loc[SNL['Target'].str.contains('FIRST HAWAIIAN'),'Target'] = 'FIRST HAWAIIAN BANK'
SNL.loc[SNL['Target'].str.contains('WASHINGTON FEDERAL'),'Target'] = 'WASHINGTON FEDERAL'

SNL.loc[SNL['Buyer']=='FIRSTMERIT','Buyer'] = 'FIRSTMERIT BANK'
SNL.loc[SNL['Buyer']=='CAPITAL ONE','Buyer'] = 'CAPITAL ONE FINANCIAL'
SNL.loc[SNL['Buyer']=='NATIONAL CITY CORP','Buyer'] = 'NATIONAL CITY CORP'
SNL.loc[SNL['Buyer']=='FIRST CHARTER CORP','Buyer'] = 'FIRST CHARTER BANK'
SNL.loc[SNL['Buyer']=='TORONTO DOMINION BANK','Buyer'] = 'TD BANK'
SNL.loc[SNL['Buyer']=='THE TORONTO DOMINION BANK','Buyer'] = 'TD BANK'
SNL.loc[SNL['Buyer']=='HIBERNIA','Buyer'] = 'HIBERNIA BANK'
SNL.loc[SNL['Buyer']=='PNC FINANCIAL SERVICES GROUP INC (THE)','Buyer'] = 'PNC BANK'
SNL.loc[SNL['Buyer']=='FLEET FINANCIAL GROUP','Buyer'] = 'FLEET BANK'
SNL.loc[SNL['Buyer']=='FLEET NATIONAL BANK','Buyer'] = 'FLEET BANK'
SNL.loc[SNL['Buyer']=='FLEET BOSTON','Buyer'] = 'FLEET BANK'
SNL.loc[SNL['Buyer']=='JP MORGAN CHASE','Buyer'] = 'CHASE BANK'
SNL.loc[SNL['Buyer']=='FIRST CHICAGO NBD','Buyer'] = 'FIRST CHICAGO BANK'
SNL.loc[SNL['Buyer']=='ALLY BANK','Buyer'] = 'ALLY FINANCIAL'
SNL.loc[SNL['Buyer']=='UBS AMERICAS','Buyer'] = 'UBS AG'
SNL.loc[SNL['Buyer']=='UBS GROUP AG','Buyer'] = 'UBS AG'
SNL.loc[SNL['Buyer']=='SYNCHRONY FINANCIAL','Buyer'] = 'SYNCHRONY BANK'
SNL.loc[SNL['Buyer']=='WEBSTER FINANCIAL','Buyer'] = 'WEBSTER BANK'
SNL.loc[SNL['Buyer']=='CULLENFROST BANKERS','Buyer'] = 'CULLENFROST BANK'
SNL.loc[SNL['Buyer']=='FNB SOUTHEAST','Buyer'] = 'FNB BANK'
SNL.loc[SNL['Buyer']=='FNB NEW MEXICO','Buyer'] = 'FNB BANK'
SNL.loc[SNL['Buyer']=='FNB FINANCIAL GROUP','Buyer'] = 'FNB BANK'
SNL.loc[SNL['Buyer']=='FNB FINANCIAL SERVICES','Buyer'] = 'FNB BANK'
SNL.loc[SNL['Buyer']=='SOUTH STATE','Buyer'] = 'SOUTH STATE BANK'
SNL.loc[SNL['Buyer']=='UMB FINANCIAL','Buyer'] = 'UMB BANK'
SNL.loc[SNL['Buyer']=='ASSOCIATED BANK GREEN BAY','Buyer'] = 'ASSOCIATED BANC'
SNL.loc[SNL['Buyer']=='HANCOCK BANK OF FLORIDA','Buyer'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Buyer']=='HANCOCK HOLDING','Buyer'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Buyer']=='HANCOCK BANK','Buyer'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Buyer']=='HANCOCK BANK MS','Buyer'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Buyer']=='HANCOCK BANK GULFPORT','Buyer'] = 'HANCOCK WHITNEY'
SNL.loc[SNL['Buyer']=='FIRST NATIONAL OF NEBRASKA','Buyer'] = 'FIRST NATIONAL BANK OF NEBRASKA'
SNL.loc[SNL['Buyer']=='FULTON FINANCIAL','Buyer'] = 'FULTON BANK'
SNL.loc[SNL['Buyer']=='ARVEST BANK GROUP','Buyer'] = 'ARVEST BANK'
SNL.loc[SNL['Buyer']=='HEARTLAND FINANCIAL USA','Buyer'] = 'HEARTLAND BANK'

SNL.loc[SNL['Buyer'].str.contains('WASHINGTON MUTUAL'),'Buyer'] = 'WASHINGTON MUTUAL'
SNL.loc[SNL['Buyer'].str.contains('ABN AMRO'),'Buyer'] = 'ABN AMRO'
SNL.loc[SNL['Buyer'].str.contains('HSBC'),'Buyer'] = 'HSBC'
SNL.loc[SNL['Buyer'].str.contains('BANC ONE'),'Buyer'] = 'BANC ONE'
SNL.loc[SNL['Buyer'].str.contains('FIRSTAR'),'Buyer'] = 'FIRSTAR'
SNL.loc[SNL['Buyer'].str.contains('FIRST NATIONWIDE'),'Buyer'] = 'FIRST NATIONWIDE BANK'
SNL.loc[SNL['Buyer'].str.contains('FIRST OF AMERICA BANK'),'Buyer'] = 'FIRST OF AMERICA BANK'
SNL.loc[SNL['Buyer'].str.contains('SIGNET BANK'),'Buyer'] = 'SIGNET BANK'
SNL.loc[SNL['Buyer'].str.contains('GREAT WESTERN'),'Buyer'] = 'GREAT WESTERN FINANCIAL'
SNL.loc[SNL['Buyer'].str.contains('CRESTAR'),'Buyer'] = 'CRESTAR FINANCIAL'
SNL.loc[SNL['Buyer'].str.contains('FIRST UNION'),'Buyer'] = 'FIRST UNION'
SNL.loc[SNL['Buyer'].str.contains('BB&T'),'Buyer'] = 'BB&T'
SNL.loc[SNL['Buyer'].str.contains('NBD'),'Buyer'] = 'NBD'
SNL.loc[SNL['Buyer'].str.contains('GOLDMAN SACHS'),'Buyer'] = 'GOLDMAN SACHS'
SNL.loc[SNL['Buyer'].str.contains('MORGAN STANLEY'),'Buyer'] = 'MORGAN STANLEY'
SNL.loc[SNL['Buyer'].str.contains('CHARLES SCHWAB'),'Buyer'] = 'CHARLES SCHWAB'
SNL.loc[SNL['Buyer'].str.contains('CAPITAL ONE'),'Buyer'] = 'CAPITAL ONE FINANCIAL'
SNL.loc[SNL['Buyer'].str.contains('BANK OF NEW YORK MELLON'),'Buyer'] = 'BANK OF NEW YORK MELLON'
SNL.loc[SNL['Buyer'].str.contains('STATE STREET'),'Buyer'] = 'STATE STREET'
SNL.loc[SNL['Buyer'].str.contains('BMO'),'Buyer'] = 'BMO BANK'
SNL.loc[SNL['Buyer'].str.contains('AMERICAN EXPRESS'),'Buyer'] = 'AMERICAN EXPRESS'
SNL.loc[SNL['Buyer'].str.contains('CITIZENS FINANCIAL'),'Buyer'] = 'CITIZENS BANK'
SNL.loc[SNL['Buyer'].str.contains('FIRST CITIZENS'),'Buyer'] = 'FIRST CITIZENS'
SNL.loc[SNL['Buyer'].str.contains('M&T'),'Buyer'] = 'M&T BANK'
SNL.loc[SNL['Buyer'].str.contains('FIFTH THIRD'),'Buyer'] = 'FIFTH THIRD BANK'
SNL.loc[SNL['Buyer'].str.contains('KEYBANK'),'Buyer'] = 'KEYCORP'
SNL.loc[SNL['Buyer'].str.contains('KEY BANK'),'Buyer'] = 'KEYCORP'
SNL.loc[SNL['Buyer'].str.contains('KEY CORP'),'Buyer'] = 'KEYCORP'
SNL.loc[SNL['Buyer'].str.contains('BARCLAYS'),'Buyer'] = 'BARCLAYS'
SNL.loc[SNL['Buyer'].str.contains('HUNTINGTON BANK'),'Buyer'] = 'HUNTINGTON BANK'
SNL.loc[SNL['Buyer'].str.contains('HUNTINGTON NATIONAL BANK'),'Buyer'] = 'HUNTINGTON BANK'
SNL.loc[SNL['Buyer'].str.contains('SANTANDER BANK'),'Buyer'] = 'SANTANDER BANK'
SNL.loc[SNL['Buyer'].str.contains('AMERIPRISE'),'Buyer'] = 'AMERIPRISE FINANCIAL'
SNL.loc[SNL['Buyer'].str.contains('RBC'),'Buyer'] = 'RBC BANK'
SNL.loc[SNL['Buyer'].str.contains('NORTHERN TRUST'),'Buyer'] = 'NORTHERN TRUST'
SNL.loc[SNL['Buyer'].str.contains('REGIONS BANK'),'Buyer'] = 'REGIONS BANK'
SNL.loc[SNL['Buyer'].str.contains('DEUTSCHE BANK'),'Buyer'] = 'DEUTSCHE BANK AG'
SNL.loc[SNL['Buyer'].str.contains('COMERICA'),'Buyer'] = 'COMERICA BANK'
SNL.loc[SNL['Buyer'].str.contains('FIRST HORIZON'),'Buyer'] = 'FIRST HORIZON BANK'
SNL.loc[SNL['Buyer'].str.contains('MIZUHO'),'Buyer'] = 'MIZUHO BANK'
SNL.loc[SNL['Buyer'].str.contains('RAYMOND JAMES'),'Buyer'] = 'RAYMOND JAMES'
SNL.loc[SNL['Buyer'].str.contains('EAST WEST'),'Buyer'] = 'EAST WEST BANK'
SNL.loc[SNL['Buyer'].str.contains('CIBC'),'Buyer'] = 'CIBC'
SNL.loc[SNL['Buyer'].str.contains('WINTRUST'),'Buyer'] = 'WINTRUST FINANCIAL'
SNL.loc[SNL['Buyer'].str.contains('PACIFIC WESTERN'),'Buyer'] = 'PACIFIC WESTERN BANK'
SNL.loc[SNL['Buyer'].str.contains('PACWEST'),'Buyer'] = 'PACIFIC WESTERN BANK'
SNL.loc[SNL['Buyer'].str.contains('STIFEL'),'Buyer'] = 'STIFEL'
SNL.loc[SNL['Buyer'].str.contains('MIDFIRST'),'Buyer'] = 'MIDFIRST BANK'
SNL.loc[SNL['Buyer'].str.contains('TIAA'),'Buyer'] = 'TIAA'
SNL.loc[SNL['Buyer'].str.contains('BANKUNITED'),'Buyer'] = 'BANKUNITED'
SNL.loc[SNL['Buyer'].str.contains('BEAL BANK'),'Buyer'] = 'BEAL BANK'
SNL.loc[SNL['Buyer'].str.contains('FIRST INTERSTATE'),'Buyer'] = 'FIRST INTERSTATE BANK'
SNL.loc[SNL['Buyer'].str.contains('SIMMONS'),'Buyer'] = 'SIMMONS BANK'
SNL.loc[SNL['Buyer'].str.contains('UNITED COMMUNITY'),'Buyer'] = 'UNITED COMMUNITY BANK'
SNL.loc[SNL['Buyer'].str.contains('SOFI'),'Buyer'] = 'SOFI BANK'
SNL.loc[SNL['Buyer'].str.contains('FIRST HAWAIIAN'),'Buyer'] = 'FIRST HAWAIIAN BANK'
SNL.loc[SNL['Buyer'].str.contains('WASHINGTON FEDERAL'),'Buyer'] = 'WASHINGTON FEDERAL'


In [13]:
#--------------------#
# Merge SNL with SOD #
#--------------------#

SNL_in_SOD = SNL.merge(banks_in_SOD.rename(columns={'name':'Target'}),on='Target')
SNL_in_SOD = SNL_in_SOD.merge(banks_in_SOD.rename(columns={'name':'Buyer'}),on='Buyer')
SNL_in_SOD = SNL_in_SOD[SNL_in_SOD['Status']=='Completed']
SNL_in_SOD = SNL_in_SOD[['Target','Buyer','Completion Date']]

In [14]:
#-------------------------#
# Manually add some deals #
#-------------------------#

SNL_in_SOD = pd.concat([SNL_in_SOD,pd.DataFrame([
    {'Target':'US TRUST','Buyer':'BANK OF AMERICA','Completion Date':datetime(2006,11,6,0,0,0)},
    {'Target':'LA SALLE BANK','Buyer':'BANK OF AMERICA','Completion Date':datetime(2007,1,1,0,0,0)},
    {'Target':'COMPASS BANK','Buyer':'BBVA BANK','Completion Date':datetime(2007,9,7,0,0,0)},
    {'Target':'STANDARD FEDERAL BANK','Buyer':'ABN AMRO','Completion Date':datetime(1996,11,21,0,0,0)},
    {'Target':'AMSOUTH BANK','Buyer':'FIRST AMERICAN NATIONAL BANK','Completion Date':datetime(1999,10,1,0,0,0)},
    {'Target':'STAR BANK','Buyer':'FIRSTAR','Completion Date':datetime(1998,11,20,0,0,0)},
    {'Target':'WELLS FARGO','Buyer':'NORWEST','Completion Date':datetime(1998,11,2,0,0,0)},
    {'Target':'NATIONSBANK','Buyer':'BANK OF AMERICA','Completion Date':datetime(1998,4,14,0,0,0)},
    {'Target':'FIRST USA BANK','Buyer':'BANC ONE','Completion Date':datetime(1997,6,27,0,0,0)},
    {'Target':'CHASE BANK','Buyer':'CHEMICAL BANK','Completion Date':datetime(1996,1,1,0,0,0)},
    {'Target':'BANK OF CALIFORNIA','Buyer':'UNION BANK','Completion Date':datetime(1996,4,1,0,0,0)},
    {'Target':'SOUTHERN NATIONAL BANK','Buyer':'BB&T','Completion Date':datetime(1995,1,1,0,0,0)},
    {'Target':'NBD','Buyer':'FIRST CHICAGO BANK','Completion Date':datetime(1995,12,1,0,0,0)},
    ])]).reset_index(drop=True)

In [15]:
#-------------#
# Export data #
#-------------#

SNL_in_SOD.to_csv('../CleanData/FDIC/0I_SNL_in_SOD.csv')
SOD.to_csv('../CleanData/FDIC/0I_SOD.csv')